In [1]:
import pandas as pd
import numpy as np
import sqlite3 as db
from datetime import datetime
import urllib
import os


con = db.connect(database="../db.sqlite")

In [2]:

df = pd.read_csv("big-mac-source-data-v2.csv")

In [3]:
df

,name,iso_a3,currency_code,local_price,dollar_ex,GDP_dollar,GDP_local,date
0,Argentina,ARG,ARS,2.50,1.00000,8709.072,8.709072e+03,2000-04-01
1,Australia,AUS,AUD,2.59,1.68000,21746.809,3.369876e+04,2000-04-01
2,Brazil,BRA,BRL,2.95,1.79000,3501.438,6.351375e+03,2000-04-01
3,Canada,CAN,CAD,2.85,1.47000,22340.553,3.319147e+04,2000-04-01
4,Switzerland,CHE,CHF,5.90,1.70000,41859.215,6.287903e+04,2000-04-01
...,...,...,...,...,...,...,...,...
1941,Uruguay,URY,UYU,255.00,41.91000,16756.344,7.291942e+05,2022-07-01
1942,United States,USA,USD,5.15,1.00000,69231.400,6.923140e+04,2022-07-01
1943,Venezuela,VEN,VES,10.00,5.67320,1685.657,NaN,2022-07-01
1944,Vietnam,VNM,VND,69000.00,23417.00000,3724.543,8.542019e+07,2022-07-01


In [6]:
##### insert countries and currencies #####

dfCurrencies = df.groupby("currency_code").first().reset_index()[["currency_code"]]
dfCurrencies = dfCurrencies.rename(columns={"currency_code": "CurrencyCode", "name":"Country"})
dfCurrencies["Name"] = dfCurrencies["CurrencyCode"]

try:
    dfCurrencies.to_sql("Currency", con,if_exists="append", index=False)
except Exception as e:
    print(e)


dfCountries = df.rename(columns={"name":"Country"}).groupby("iso_a3").first().reset_index()[["iso_a3", "Country", "currency_code"]]
dfCountries = dfCountries.rename(columns={"iso_a3": "CountryCode", "Country": "Name", "currency_code": "Currency_id"})

try:
    dfCountries.to_sql("Country", con,if_exists="append", index=False)
except Exception as e:
    print(e)

UNIQUE constraint failed: Currency.CurrencyCode
UNIQUE constraint failed: Country.CountryCode


In [7]:
df['dollar_price'] = df['local_price'] * df['dollar_ex']

In [8]:
df

,name,iso_a3,currency_code,local_price,dollar_ex,GDP_dollar,GDP_local,date,dollar_price
0,Argentina,ARG,ARS,2.50,1.00000,8709.072,8.709072e+03,2000-04-01,2.500000e+00
1,Australia,AUS,AUD,2.59,1.68000,21746.809,3.369876e+04,2000-04-01,4.351200e+00
2,Brazil,BRA,BRL,2.95,1.79000,3501.438,6.351375e+03,2000-04-01,5.280500e+00
3,Canada,CAN,CAD,2.85,1.47000,22340.553,3.319147e+04,2000-04-01,4.189500e+00
4,Switzerland,CHE,CHF,5.90,1.70000,41859.215,6.287903e+04,2000-04-01,1.003000e+01
...,...,...,...,...,...,...,...,...,...
1941,Uruguay,URY,UYU,255.00,41.91000,16756.344,7.291942e+05,2022-07-01,1.068705e+04
1942,United States,USA,USD,5.15,1.00000,69231.400,6.923140e+04,2022-07-01,5.150000e+00
1943,Venezuela,VEN,VES,10.00,5.67320,1685.657,NaN,2022-07-01,5.673200e+01
1944,Vietnam,VNM,VND,69000.00,23417.00000,3724.543,8.542019e+07,2022-07-01,1.615773e+09


In [10]:
### insert bigmac values ###



dfbm = df[["iso_a3", "local_price", "dollar_price", "date"]].rename(columns={"iso_a3":"Country_id", "date": "Date", "local_price":"LocalPrice","dollar_price":"USDPrice"})
dfbm["Source"] = "Economist"
try:
    dfbm.to_sql("BigMacPrice", con,if_exists="append", index=False)
except Exception as e:
    print(e)



UNIQUE constraint failed: BigMacPrice.Country_id, BigMacPrice.Date


In [ ]:
#delete from BigMacPrice where id in (
#select minrowid from(
#SELECT rowId, strftime('%Y-%m-%d',Date) as date, country_id, Source, count(*), min(id) minrowid
#from BigMacPrice
#group by country_id, source, strftime('%Y-%m-%d',Date)
#having count(*) > 1))